# Reaction annotation
Using MetaNetX reac_xref.tsv (can  be downloaded from [here](https://www.metanetx.org/mnxdoc/mnxref.html)).

In [1]:
import re
from collections import defaultdict
from pathlib import Path

import cobra
from datatable import dt, f, join, update

In [2]:
ROOT = Path.cwd().parent
model_file = str(ROOT / "iMENI452.xml")

In [3]:
model = cobra.io.read_sbml_model(model_file)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Gather all prepared reactions in a dataframe matched to their identifiers in the model.

In [4]:
reacs_prepared = [reac.id for reac in model.reactions]

In [5]:
reacs_prepared[:5]

['ALAD_L', 'ASPTA', 'ARGSS', 'ARGSL', 'ADSS']

These are bigg reactions which can be already annotated.

In [7]:
for reac in model.reactions:
    reac.annotation["bigg.reaction"] = reac.id

In [8]:
reacs_prepared = dt.Frame(
    key=reacs_prepared
)

In [9]:
reacs_prepared.head()

,key
,▪▪▪▪
0,ALAD_L
1,ASPTA
2,ARGSS
3,ARGSL
4,ADSS
5,ADSL1r
6,ASPT
7,ASPCT
8,ASP1DC


Now that we have the model reactions in a data frame, let's prepare the MNX dataset.

In [11]:
mnx = dt.fread(str(ROOT / "reac_xref.tsv"), skip_to_line=352)

In [12]:
mnx.head()

,#source,ID,description
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,EMPTY,EMPTY,Empty equation
1,bigg.reaction:CRBNTD,EMPTY,H2CO3 dissociation||1 biggM:h2co3@biggC:x = 1 bigg…
2,bigg.reaction:DNADRAIN,EMPTY,Dna sink transport reaction|| =
3,bigg.reaction:H2CO3D2,EMPTY,Carboxylic acid dissociation||1 biggM:h2co3@biggC:…
4,bigg.reaction:H2CO3D2m,EMPTY,"Carboxylic acid dissociation, mitochondrial||1 big…"
5,bigg.reaction:HMR_5409,EMPTY,HMR 5409||1 biggM:h@biggC:e + 1 biggM:hco3@biggC:e…
6,bigg.reaction:NH3c,EMPTY,Ammomnium proton dissociation||1 biggM:nh4@biggC:c…
7,bigg.reaction:NH4DISg,EMPTY,Nh4 Dissociation||1 biggM:h@biggC:x + 1 biggM:nh3@…
8,bigg.reaction:RE2594C,EMPTY,RE2594||1 biggM:h@biggC:c + 1 biggM:CE2949@biggC:c…


In [13]:
mnx.tail()

,#source,ID,description
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,biggR:GALNACT2g,MNXR99997,Uridine diphosphoacetylgalactosamine-chondroitin a…
1,biggR:R_GALNACT2g,MNXR99997,secondary/obsolete/fantasy identifier
2,bigg.reaction:GALNACT3g,MNXR99998,Uridine diphosphoacetylgalactosamine-chondroitin a…
3,bigg.reaction:R_GALNACT3g,MNXR99998,secondary/obsolete/fantasy identifier
4,biggR:GALNACT3g,MNXR99998,Uridine diphosphoacetylgalactosamine-chondroitin a…
5,biggR:R_GALNACT3g,MNXR99998,secondary/obsolete/fantasy identifier
6,bigg.reaction:GALNACT4g,MNXR99999,Uridine diphosphoacetylgalactosamine-chondroitin a…
7,bigg.reaction:R_GALNACT4g,MNXR99999,secondary/obsolete/fantasy identifier
8,biggR:GALNACT4g,MNXR99999,Uridine diphosphoacetylgalactosamine-chondroitin a…


In [14]:
mnx.names = ["key", "mnx", "description"]

## MNX space

The first step is to map the BiGG reaction identifiers to MXN.

In [16]:
bigg_mnx = mnx[
    dt.re.match(f.key, "^bigg\.reaction:.+"), {"key": f.key[14:], "mnx": f.mnx}
]

In [17]:
bigg_mnx

,key,mnx
,▪▪▪▪,▪▪▪▪
0,CRBNTD,EMPTY
1,DNADRAIN,EMPTY
2,H2CO3D2,EMPTY
3,H2CO3D2m,EMPTY
4,HMR_5409,EMPTY
5,NH3c,EMPTY
6,NH4DISg,EMPTY
7,RE2594C,EMPTY
8,R_CRBNTD,EMPTY


In [18]:
bigg_mnx.key = "key"

Join by bigg id in the model, so that we take only the reactions of interest.

In [21]:
reacs_prepared = reacs_prepared[:, :, join(bigg_mnx)]

In [23]:
reacs_prepared.nrows

813

In [28]:
reacs_prepared[dt.isna(f.mnx), :].nrows

253

In [24]:
len(model.reactions)

813

In [25]:
for reac in model.reactions:
    matched = reacs_prepared[f.key == reac.id, "mnx"]
    if matched.nrows:
        reac.annotation["metanetx.reaction"] = matched[0, 0]

In [26]:
len([1 for reac in model.reactions if "metanetx.reaction" in reac.annotation if reac.annotation["metanetx.reaction"] is not None])

560

In [29]:
for reac in model.reactions:
    if "metanetx.reaction" in reac.annotation:
        if reac.annotation["metanetx.reaction"] is None:
            del reac.annotation["metanetx.reaction"]

In [30]:
len(model.reactions)

813

## Annotating BiGG

In [31]:
mnx["db"] = "mnx"

In [32]:
def get_id_db(id_base: str) -> (str, str):
    id_base = id_base.split(":", maxsplit=1)
    if len(id_base) > 1:
        db, id = id_base
    else:
        db, id = "mnx", id_base[0]
    return id, db

In [33]:
ids_dbs = [get_id_db(mnx[i, "key"]) for i in range(mnx.nrows)]
ids = [el[0] for el in ids_dbs]
dbs = [el[1] for el in ids_dbs]
mnx["key"] = dt.Frame(ids)
mnx["db"] = dt.Frame(dbs)

In [34]:
mnx

,key,mnx,description,db
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,EMPTY,EMPTY,Empty equation,mnx
1,CRBNTD,EMPTY,H2CO3 dissociation||1 biggM:h2co3@biggC:x = 1 bigg…,bigg.reaction
2,DNADRAIN,EMPTY,Dna sink transport reaction|| =,bigg.reaction
3,H2CO3D2,EMPTY,Carboxylic acid dissociation||1 biggM:h2co3@biggC:…,bigg.reaction
4,H2CO3D2m,EMPTY,"Carboxylic acid dissociation, mitochondrial||1 big…",bigg.reaction
5,HMR_5409,EMPTY,HMR 5409||1 biggM:h@biggC:e + 1 biggM:hco3@biggC:e…,bigg.reaction
6,NH3c,EMPTY,Ammomnium proton dissociation||1 biggM:nh4@biggC:c…,bigg.reaction
7,NH4DISg,EMPTY,Nh4 Dissociation||1 biggM:h@biggC:x + 1 biggM:nh3@…,bigg.reaction
8,RE2594C,EMPTY,RE2594||1 biggM:h@biggC:c + 1 biggM:CE2949@biggC:c…,bigg.reaction


In [35]:
for reac in model.reactions:
    if "metanetx.reaction" in reac.annotation:
        kv = mnx[f.mnx == reac.annotation["metanetx.reaction"], ["key", "db"]].to_list()
        data_dict = defaultdict(list)
        for k, v in zip(kv[1], kv[0]):
            data_dict[k].append(v)
        for k, v in data_dict.items():
            data_dict[k] = v if len(v) > 1 else v[0]
        reac.annotation = data_dict

In [36]:
dbs = dt.unique(mnx["db"]).to_list()[0]

In [37]:
dbs

['bigg.reaction',
 'biggR',
 'kegg.reaction',
 'keggR',
 'metacyc.reaction',
 'metacycR',
 'mnx',
 'rhea',
 'rheaR',
 'sabiork.reaction',
 'sabiorkR',
 'seed.reaction',
 'seedR']

In [38]:
for db in dbs:
    print(
        f"{db} annotated: {len([1 for reac in model.reactions if db in reac.annotation])}"
    )

bigg.reaction annotated: 813
biggR annotated: 560
kegg.reaction annotated: 286
keggR annotated: 286
metacyc.reaction annotated: 303
metacycR annotated: 303
mnx annotated: 0
rhea annotated: 292
rheaR annotated: 292
sabiork.reaction annotated: 232
sabiorkR annotated: 232
seed.reaction annotated: 424
seedR annotated: 424


"metanetx.reaction" annotations were removed while computing the others. Add them back

In [40]:
for reac in model.reactions:
    matched = reacs_prepared[f.key == reac.id, "mnx"]
    if matched.nrows:
        if matched[0,0] is not None:
            reac.annotation["metanetx.reaction"] = matched[0, 0]

In [41]:
len([1 for reac in model.reactions if "metanetx.reaction" in reac.annotation])

560

In [43]:
for reac in model.reactions:
    if reac.annotation is None:
        reac.annotation == {}
    to_del = [k for k, v in reac.annotation.items() if v is None]
    for k in to_del:
        del reac.annotation[k]
        del reac.annotation["mnx"]

In [44]:
cobra.io.write_sbml_model(model, model_file)

## SBO terms

SBO:0000176 represents the term 'biochemical reaction'. Every metabolic reaction that is not a transport or boundary reaction should be annotated with this.

In [45]:
from memote.support.helpers import find_transport_reactions

In [46]:
transport_reactions = find_transport_reactions(model)

In [47]:
for reac in model.reactions:
    reac.annotation["sbo"] = (
        "SBO:0000185" if reac in transport_reactions else "SBO:0000176"
    )

Now, biomass reaction.

In [49]:
model.reactions.biomass.annotation["sbo"] = "SBO:0000629"

SBO:0000627 represents the term 'exchange reaction'.

In [50]:
for ex in model.exchanges:
    ex.annotation["sbo"] = "SBO:0000627"

SBO:0000243 represents the term 'gene'. Every gene should be annotated with this.

In [51]:
for gene in model.genes:
    gene.annotation["sbo"] = "SBO:0000243"

In [52]:
cobra.io.write_sbml_model(model, model_file)